# Ex 2.calitatea localizarii corecte a textului in imagine

In [3]:
import cv2
import pytesseract
from pytesseract import Output

# Configurez calea catre executabilul Tesseract (Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Incarc imaginea
image = cv2.imread('image.jpeg')
# Selectează regiunea manual
roi = cv2.selectROI("Selecteaza Ground Truth", image, fromCenter=False, showCrosshair=True)

# Extrage coordonatele (x1, y1, lățime, înălțime)
x1, y1, w, h = roi
x2, y2 = x1 + w, y1 + h

ground_truth_box = (x1, y1, x2, y2)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Convertesc imaginea in tonuri de gri
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Aplic un filtru de thresholding (binarizare)
_, binary = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

# Salvez imaginea preprocesata
cv2.imwrite('preprocessed_image.jpg', binary)

# Incarc imaginea preprocesata
image = cv2.imread('preprocessed_image.jpg')

# Detectez regiunile de text folosind Tesseract
data = pytesseract.image_to_data(image, output_type=Output.DICT)

# Functie pentru calculul IoU
def calculate_iou(box1, box2):
    """
    Calculeaza Intersection over Union (IoU) intre doua dreptunghiuri.
    box1 și box2 sunt tuple de forma (x1, y1, x2, y2).
    """
    # Coordonatele intersectiei
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Aria intersectiei
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)

    # Aria celor doua dreptunghiuri
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Aria uniunii
    union_area = box1_area + box2_area - inter_area

    # IoU
    iou = inter_area / union_area if union_area != 0 else 0
    return iou



# Parcurg toate regiunile detectate și calculez IoU pentru fiecare
iou_values = []
for i in range(len(data['text'])):
    if int(data['conf'][i]) > 0:  # Filtrez regiunile fara text detectat
        detected_box = (
            data['left'][i],
            data['top'][i],
            data['left'][i] + data['width'][i],
            data['top'][i] + data['height'][i]
        )
        iou = calculate_iou(ground_truth_box, detected_box)
        iou_values.append(iou)

# Returnez cea mai mare valoare IoU (cea mai buna suprapunere)
max_iou = max(iou_values) if iou_values else 0
print("Intersection over Union (IoU):", max_iou)

# Desenez regiunile de text detectate
for i in range(len(data['text'])):
    if int(data['conf'][i]) > 0:
        (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Salvez imaginea cu regiunile de text detectate
cv2.imwrite('detected_text.jpg', image)
print("Imaginea cu regiunile de text detectate a fost salvată ca 'detected_text.jpg'.")


Intersection over Union (IoU): 0.276566757493188
Imaginea cu regiunile de text detectate a fost salvată ca 'detected_text.jpg'.
